In [ ]:
!pip install --upgrade asyncpraw


In [9]:
import asyncpraw
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics
from transformers import AutoTokenizer
import pandas as pd

In [10]:
CLIENT_ID = 'Exr50dzRIEGoBbIy0VHdLA'
CLIENT_SECRET = 'k-DgFJipW5NrbLiDeKbbCzX5xeLEIg'
USER_AGENT = 'test script'

reddit = asyncpraw.Reddit(client_id=CLIENT_ID,
                     client_secret=CLIENT_SECRET,
                     user_agent=USER_AGENT,
                     password="Badboy@2903",
                     username="AgitatedWinner3627",
                     )

In [11]:
# # Define function to fetch subreddit data
# async def fetch_subreddit_data(subreddit_name, num_posts):
#     subreddit = await reddit.subreddit(subreddit_name)
#     posts = []
#     async for post in subreddit.hot(limit=num_posts):
#         posts.append((post.title, post.link_flair_text))
#     return posts

# # Fetch data from subreddit
# subreddit_name = 'machinelearning'
# num_posts = 1000

# # Fetch posts from subreddit
# posts = await fetch_subreddit_data(subreddit_name, num_posts)

In [12]:
indiaposts = pd.read_csv('/content/reddit_data.csv')

In [13]:
indiaposts.keys()

Index(['Flair', 'Post'], dtype='object')

In [14]:
indiaposts['Flair'][100]

'Politics'

In [15]:
labelcount = {}
for p in list(indiaposts['Flair']):
  labelcount[p] = labelcount.get(p, 0) + 1

for i in labelcount:
  print(i, ": ", labelcount[i])

posts = [[indiaposts['Post'][i], indiaposts['Flair'][i]] for i in range(len(indiaposts))]

Politics :  150
Coronavirus :  150
AskIndia :  150
Non-Political :  150
Policy/Economy :  148
Scheduled :  136
Business/Finance :  143
Science/Technology :  144
Food :  125
Photography :  131


In [16]:
# Preprocess data
batch_size = 32
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# posts = posts[:(len(posts)//batch_size)*batch_size]
labels = [post[1] for post in posts]
uniquelabels = list(set(labels))
labeldict = {uniquelabels[i]: i for i in range(len(uniquelabels))}
labels = [labeldict[post[1]] for post in posts]
texts = [post[0] for post in posts]

# Split data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Apply encoding and onehot
train_texts = tokenizer([t for t in train_texts], truncation=True, padding=True, add_special_tokens=True)
test_texts = tokenizer([t for t in test_texts], truncation=True, padding=True, add_special_tokens=True)
# train_labels = torch.nn.functional.one_hot(torch.Tensor(train_labels).to(torch.int64), len(uniquelabels))
# test_labels = torch.nn.functional.one_hot(torch.Tensor(test_labels).to(torch.int64), len(uniquelabels))
train_labels = torch.Tensor(train_labels).to(torch.int64)
test_labels = torch.Tensor(test_labels).to(torch.int64)


# Define dataset class
class RedditDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels):
        super().__init__()
        self.texts = texts
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.texts.items()}
        item['label'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset instances
train_dataset = RedditDataset(train_texts, train_labels)
test_dataset = RedditDataset(test_texts, test_labels)

# Define data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [17]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# Define transformer-based classification model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(uniquelabels))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()
trainloss = []
for epoch in range(30):  # Just for demonstration, increase for actual training

    batchloss = 0
    batchiter = 0
    for i, batch in enumerate(train_loader):
        tinput_ids = batch['input_ids'].to(device)
        tattention_mask = batch['attention_mask'].to(device)
        tlabels = batch['label'].to(device)
        optimizer.zero_grad()
        outputs = model(input_ids=tinput_ids, attention_mask=tattention_mask)
        loss = criterion(outputs.logits, tlabels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        batchloss+=loss.item()
        batchiter+=1
    print("Epoch: ", epoch+1, " loss: ", batchloss/batchiter)
    trainloss.append((epoch+1, batchloss/batchiter))

ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7abed9e15840>
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch:  1  loss:  2.272140575779809
Epoch:  2  loss:  2.0289569430881076
Epoch:  3  loss:  1.7420290344291263
Epoch:  4  loss:  1.4341548681259155
Epoch:  5  loss:  1.1743361237976286
Epoch:  6  loss:  0.9696686516205469
Epoch:  7  loss:  0.8032567724585533
Epoch:  8  loss:  0.6659962187210718
Epoch:  9  loss:  0.5533320092492633
Epoch:  10  loss:  0.4551495479212867
Epoch:  11  loss:  0.3756396257215076
Epoch:  12  loss:  0.319419471340047
Epoch:  13  loss:  0.26742136602600414
Epoch:  14  loss:  0.22213403239018387
Epoch:  15  loss:  0.1969398818910122
Epoch:  16  loss:  0.17939192863802114
Epoch:  17  loss:  0.16526386017600694
Epoch:  18  loss:  0.14993751359482607
Epoch:  19  loss:  0.13954791768143573
Epoch:  20  loss:  0.12796557808501852
Epoch:  21  loss:  0.1269758182267348
Epoch:  22  loss:  0.12749363626870844
Epoch:  23  loss:  0.1134454310975141
Epoch:  24  loss:  0.10902719013392925
Epoch:  25  loss:  0.11093060739545359
Epoch:  26  loss:  0.10318663871536653
Epoch:  27  

In [ ]:
# Evaluation
model.eval()
predicted_labels = []
true_labels = []
with torch.no_grad():
    for batch in test_loader:
        tinput_ids = batch['input_ids'].to(device)
        tattention_mask = batch['attention_mask'].to(device)
        tlabels = batch['label'].to(device)

        outputs = model(input_ids=tinput_ids, attention_mask=tattention_mask)
        _, predicted = torch.max(outputs.logits, 1)
        predicted_labels.extend(predicted.cpu().numpy())
        true_labels.extend(tlabels.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f'Accuracy: {accuracy:.2f}')

In [20]:
def scoring(y_test, y_predict):
    acc = metrics.accuracy_score(y_test, y_predict)
    f1_score = metrics.f1_score(y_test, y_predict, average='macro')
    precision = metrics.precision_score(y_test,y_predict,average='macro')
    recall = metrics.recall_score(y_test,y_predict,average='macro')
    print('the classifier has\n f1_score: {}\n accuracy: {}\n precision:{}\n recall: {}'.format(f1_score,acc,precision,recall))

In [ ]:
scoring(true_labels, predicted_labels)